In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
import json
from keras.models import model_from_json

In [2]:
training = np.genfromtxt('/path/to/Sentiment Analysis Dataset.csv', delimiter=',', skip_header=1, usecols=(1, 3),max_rows=100000, dtype=None)

train_x = [x[1] for x in training]
# index all the sentiment labels
train_y = np.asarray([x[0] for x in training])

/home/exacon03/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


In [3]:
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout

def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    op_units, op_activation = 1,'sigmoid'
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units, activation=op_activation))
    return model

In [4]:
kwargs = {
            'ngram_range': (1,2),  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': 'word',  # Split text into word tokens.
            'min_df':  1,
            'max_df': 1.0,
}
vectorizer = TfidfVectorizer(**kwargs)
# Learn vocabulary from training texts and vectorize training texts.
train_x = vectorizer.fit_transform(train_x)
 # Select top 'k' of the vectorized features.
selector = SelectKBest(chi2, k=min(20000, train_x.shape[1]))
selector.fit(train_x, train_y)
# print(train_x)
train_x = selector.transform(train_x).astype('float32')

In [5]:
model=mlp_model(2,32,0.3,(20000,),2)

In [6]:
model.compile(loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

model.fit(train_x, train_y,
  batch_size=512,
  epochs=20,
  verbose=1,
  validation_split=0.2,
  callbacks = [EarlyStopping(monitor='val_loss', patience=1)],     
  shuffle=True)

Train on 80000 samples, validate on 20000 samples
Epoch 1/20
80000/80000 [==============================] - 19s 238us/step - loss: 0.6508 - acc: 0.6234 - val_loss: 0.5888 - val_acc: 0.7301
Epoch 2/20
80000/80000 [==============================] - 19s 233us/step - loss: 0.5645 - acc: 0.7266 - val_loss: 0.5216 - val_acc: 0.7663
Epoch 3/20
80000/80000 [==============================] - 19s 232us/step - loss: 0.5194 - acc: 0.7490 - val_loss: 0.4908 - val_acc: 0.7752
Epoch 4/20
80000/80000 [==============================] - 18s 229us/step - loss: 0.4916 - acc: 0.7675 - val_loss: 0.4741 - val_acc: 0.7827
Epoch 5/20
80000/80000 [==============================] - 18s 228us/step - loss: 0.4715 - acc: 0.7796 - val_loss: 0.4625 - val_acc: 0.7880
Epoch 6/20
80000/80000 [==============================] - 18s 228us/step - loss: 0.4580 - acc: 0.7868 - val_loss: 0.4552 - val_acc: 0.7924
Epoch 7/20
80000/80000 [==============================] - 19s 232us/step - loss: 0.4472 - acc: 0.7923 - val_loss: 0.

In [12]:
# save model as json file
model_json = model.to_json()
with open('VectorModel.json', 'w') as json_file:
    json_file.write(model_json)

#save weights of models separately
model.save_weights('VectorModel.h5')

In [15]:
# read in your saved model structure
json_file = open('VectorModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
# and create a model from that
model = model_from_json(loaded_model_json)
# and weight your nodes with your saved values
model.load_weights('VectorModel.h5')

In [ ]:
labels= ['Negative','Positive']
while 1:
    evalSentence =input('Input a sentence to be evaluated, or Enter to quit: ')
    evalSentence = [evalSentence]
    if len(evalSentence) == 0:
        break

    # format your input for the neural net
    tx = vectorizer.transform(evalSentence)
    tx = selector.transform(tx).astype('float32')
    #run model
    pred = model.predict(tx)
    print("%s sentiment; %f%% confidence" % (labels[int(np.round(pred[0][0]))], np.abs(pred[0][0]-0.5)*200))

Input a sentence to be evaluated, or Enter to quit: helloi
Positive sentiment; 29.558897% confidence
Input a sentence to be evaluated, or Enter to quit: this is bad
Negative sentiment; 98.415571% confidence
Input a sentence to be evaluated, or Enter to quit: hello
Positive sentiment; 92.687285% confidence
Input a sentence to be evaluated, or Enter to quit: hello!
Positive sentiment; 92.687285% confidence
